In [1]:
import sys, os, time, cv2
from os import listdir
import numpy as np

timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../.')) 

from makeStableBaselinesILEnv import makeStableBaselinesILEnv
from randomTestSBWrappers import showObs

In [2]:
 # Show files in folder
base_path = os.path.abspath('')
trajRecFolder = os.path.join(base_path, "../../test/data/Discrete/Normal")
#trajRecFolder = os.path.join(base_path, "../../test/data/Discrete/HC")
#trajRecFolder = os.path.join(base_path, "../../test/data/MultiDiscrete/Normal")
#trajRecFolder = os.path.join(base_path, "../../test/data/MultiDiscrete/HC")
trajectoriesFiles = [os.path.join(trajRecFolder, f) for f in listdir(trajRecFolder) if os.path.isfile(os.path.join(trajRecFolder, f))]
print(trajectoriesFiles)

['/home/apalmas/Applications/Diambra/diambraengine/stableBaselines/tests/../../test/data/Discrete/Normal/mod0_P1_Hayabusa_diff3_rew-2.51_2021-06-18_16-29-19', '/home/apalmas/Applications/Diambra/diambraengine/stableBaselines/tests/../../test/data/Discrete/Normal/mod0_P1P2_TinaAyane_rew-1.038_2021-06-18_16-38-01']


In [3]:
gameId = "doapp"
diambraILKwargs = {}
diambraILKwargs["trajFilesList"] = trajectoriesFiles
diambraILKwargs["totalCpus"] = 1

hardCore = True if trajRecFolder[-2:] == "HC" else False
idxList = [0]

In [4]:
# Additional obs key list
keyToAdd = []
keyToAdd.append("actionsBuf") # env.actBufLen*(env.n_actions[0]+env.n_actions[1])

if gameId != "tektagt":                                                         
    keyToAdd.append("ownHealth")   # 1                                            
    keyToAdd.append("oppHealth")   # 1                                                
else:                                                                           
    keyToAdd.append("ownHealth1")    # 1                                             
    keyToAdd.append("ownHealth2")    # 1                                             
    keyToAdd.append("oppHealth1")    # 1                                              
    keyToAdd.append("oppHealth2")    # 1  
    keyToAdd.append("ownActiveChar") # 1
    keyToAdd.append("oppActiveChar") # 1
    
keyToAdd.append("ownPosition")     # 1
keyToAdd.append("oppPosition")     # 1
keyToAdd.append("stage")           # 1
keyToAdd.append("ownChar")       # len(env.charNames)
keyToAdd.append("oppChar")       # len(env.charNames)

In [5]:
envId = "Test"
env = makeStableBaselinesILEnv(envId, diambraILKwargs, timeDepSeed, hardCore=hardCore, 
                               keyToAdd=keyToAdd, useSubprocess=True)


Using Discrete action space


In [6]:
observation = env.reset()[0]
env.render(mode="human")

One player RL trajectory


KeyError: 'ownChar'

In [ ]:
nChars = env.get_attr("nChars")[0]
charNames = env.get_attr("charNames")[0]
nActions = env.get_attr("nActions")[0]
actBufLen = env.get_attr("actBufLen")[0]
playerSide = env.get_attr("playerSide")[0]

In [ ]:
limAct = [None, None]
for idx in range(2):
    limAct[idx] = [actBufLen*nActions[0], 
                   actBufLen*nActions[0] + actBufLen*nActions[1]]

In [ ]:
env.env_method("trajSummary")

In [ ]:
showObs(observation, gameId, actBufLen, 0, True, charNames, limAct, hardCore, idxList)

In [ ]:
cumulativeEpRew = 0.0
cumulativeEpRewAll = []

maxNumEp = 10
currNumEp = 0

procIdx = 0

while currNumEp < maxNumEp:
       
    dummyActions = [0 for i in range(diambraILKwargs["totalCpus"])]
    observation, reward, done, info = env.step(dummyActions)
    
    if np.any(env.get_attr("exhausted")):
        break
    
    env.render(mode="human")
    
    observation = observation[procIdx]
    reward = reward[procIdx]
    done = done[procIdx]
    action = info[procIdx]["action"]
    print("Action = ", action)
    print("Reward = ", reward)
    #if done:
    #    observation = info[procIdx]["terminal_observation"]
    
    # Visualize observations content
    showObs(observation, gameId, env.actBufLen, 0, True, env.charNames, limAct, hardCore, idxList)
 
    cumulativeEpRew += reward
    
    if done:
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeEpRew = 0.0
        
print("All ep. rewards =", cumulativeEpRewAll)   
print("Mean cumulative reward =", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward =", np.std(cumulativeEpRewAll))       